<a href="https://colab.research.google.com/github/antnewman/examples-colab-neo4j-films/blob/main/example_colab_neo4j_films.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neo4j with Google Colab Example 

# Introduction

This notebook covers the following:
- How to connect a Google Colab notebook with a graph database using the Neo4J sandbox.
- Access nodes and relationships.
- Perform operations over these nodes and relationships.
- Run some basic Cypher queries and retrieve results in Dataframe.
- Add further nodes and relationships from to the graph database from the Colab.

# Setup

Check Python version and install appropriate minicoda.

In [1]:
!which python # should return /usr/local/bin/python
!python --version

/usr/local/bin/python
Python 3.6.12 :: Anaconda, Inc.


In [2]:
!echo $PYTHONPATH # If /env/python then unset the path, becaue this directory doesn't seeem to exist within the Google Colab file system 

/env/python


Unset pythonpath variable before installing Miniconda as it can cause problems if there are packages installed and accessible via directories included in the PYTHONPATH that are not compatible with the version of Python included with Miniconda.

In [3]:
%env PYTHONPATH=

env: PYTHONPATH=


**Installing Miniconda**

Download the installer script for the appropriate version of Miniconda and install it into /usr/local. 

Installing directly into /usr/local, rather than into the default location ~/miniconda3, insures that Conda and all its required dependencies will be automatically available for use within Google Colab.

In [4]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1.20170329-h6b74fdf_2 ...
installing: readline-7.0-ha6073c6_4 ...
installing: sqlite-3.23.1-he433501_0 ...
installing: asn1crypto-0.24.0-py36_0 ...
installing: certifi-2018.4.16-py36_0 ...
installing: chardet-3.0.4-py36h0f667ec_1 ...
installing: idna-2.6-py36h82fb2a8_1 ...
installing: pycosat-0.6.3-py36h0a5515d_0 ...
installing: pycparser-2.18-py36hf9f622e_1 ...
installing: pysocks-1.6.8-py36_0 ...
installing: ruamel_yaml-0.15.37-py36h14c3975_2 ...
installing: six-1.11

--2021-02-26 16:13:24--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh [following]
--2021-02-26 16:13:24--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh.2’

     0K .......... .......... .......... .......... ..........  0% 14.3M 4s
    50K .......... .......... .......... .......... ..........  

Verify that:
- the conda executable is available
- the version is correct
- Installing has not impacted the python executable
- Verify which version of Python has been install by Miniconda

In [5]:
!which conda # should return /usr/local/bin/conda
!conda --version # should return 4.5.4
!which python # still returns /usr/local/bin/python
!python --version # now returns Python 3.6.5 :: Anaconda, Inc.

/usr/local/bin/conda
conda 4.5.4
/usr/local/bin/python
Python 3.6.5 :: Anaconda, Inc.


**Updating Conda**

We need to update Conda and its dependencies to their most recent versions without updating Python to 3.7 for the ipython-cypher compatibility.

In [6]:
%%bash

# Updates Conda to the most recent version, but hold Python version fixed at <3.7 to be compatible with ipython-cypher
conda install --channel defaults conda python=3.6 --yes

# Updates all of Conda’s dependencies to their most recent versions.
conda update --channel defaults --all --yes 

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs: 
    - conda
    - python=3.6


The following packages will be UPDATED:

    ca-certificates: 2018.03.07-0            --> 2021.1.19-h06a4308_0    
    certifi:         2018.4.16-py36_0        --> 2020.12.5-py36h06a4308_0
    cffi:            1.11.5-py36h9745a5d_0   --> 1.14.5-py36h261ae71_0   
    conda:           4.5.4-py36_0            --> 4.9.2-py36h06a4308_0    
    cryptography:    2.2.2-py36h14c3975_0    --> 3.3.1-py36h3c74f83_1    
    libedit:         3.1.20170329-h6b74fdf_2 --> 3.1.20191231-h14c3975_1 
    libffi:          3.2.1-hd88cf55_4        --> 3.3-he6710b0_2          
    libgcc-ng:       7.2.0-hdf63c60_3        --> 9.1.0-hdf63c60_0        
    libstdcxx-ng:    7.2.0-hdf63c60_3        --> 9.1.0-hdf63c60_0        
    ncurses:         6.1-hf484d3e_0          --> 6.2-he6710b0_1          
    openssl:         1.0.2o-h20670df_0       --> 1.1.1j-h27cfd23_


# >>>>>>>>>>>>>>>>>>>>>> ERROR REPORT <<<<<<<<<<<<<<<<<<<<<<

    Traceback (most recent call last):
      File "/usr/local/lib/python3.6/site-packages/conda/exceptions.py", line 1079, in __call__
        return func(*args, **kwargs)
      File "/usr/local/lib/python3.6/site-packages/conda/cli/main.py", line 84, in _main
        exit_code = do_call(args, p)
      File "/usr/local/lib/python3.6/site-packages/conda/cli/conda_argparse.py", line 83, in do_call
        return getattr(module, func_name)(args, parser)
      File "/usr/local/lib/python3.6/site-packages/conda/cli/main_update.py", line 20, in execute
        install(args, parser, 'update')
      File "/usr/local/lib/python3.6/site-packages/conda/cli/install.py", line 116, in install
        if context.use_only_tar_bz2:
      File "/usr/local/lib/python3.6/site-packages/conda/base/context.py", line 714, in use_only_tar_bz2
        import conda_package_handling.api
      File "/usr/local/lib/python3.6/site-packages/conda_package_

Check versions of conda and python.

In [7]:
!conda --version # now returns 4.9.2
!python --version # now returns Python 3.6.12 :: Anaconda, Inc.

conda 4.9.2
Python 3.6.12 :: Anaconda, Inc.


**Append to** *sys.path* 

We need to add the directory, to which Conda will install packages to the list of directories that Python will search when looking for modules to import.

Check the current list of dirs that Python will search by inspecting the *sys.path*.

In [8]:
import sys
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython']

Pre-installed packages are in dist-packages. Conda installed packages are in site-packages.

In [9]:
!ls /usr/local/lib/python3.7/dist-packages

absl
absl_py-0.10.0.dist-info
alabaster
alabaster-0.7.12.dist-info
albumentations
albumentations-0.1.12.dist-info
altair
altair-4.1.0.dist-info
apiclient
appdirs-1.4.4.dist-info
appdirs.py
apt
apt_inst.cpython-37m-x86_64-linux-gnu.so
apt_inst.pyi
apt_pkg.cpython-37m-x86_64-linux-gnu.so
apt_pkg.pyi
aptsources
argon2
argon2_cffi-20.1.0.dist-info
asgiref
asgiref-3.3.1.dist-info
astor
astor-0.8.1.dist-info
astropy
astropy-4.1.dist-info
astunparse
astunparse-1.6.3.dist-info
async_generator
async_generator-1.10.dist-info
atari_py
atari_py-0.2.6.dist-info
atomicwrites
atomicwrites-1.4.0.dist-info
attr
attrs-20.3.0.dist-info
audioread
audioread-2.1.9.dist-info
autograd
autograd-1.3.dist-info
babel
Babel-2.9.0.dist-info
backcall
backcall-0.2.0.dist-info
beautifulsoup4-4.6.3.dist-info
bin
bleach
bleach-3.3.0.dist-info
blis
blis-0.4.1.dist-info
bokeh
bokeh-2.1.1.dist-info
bottleneck
Bottleneck-1.3.2.dist-info
branca
branca-0.4.2.dist-info
bs4
bs4-0.0.1.dist-info
bson
cachecontrol
CacheControl-0.1

In [10]:
import sys
_ = (sys.path.append("/usr/local/lib/python3.7/site-packages"))

Note that the dist-packages directory containing the pre-installed Colab packages appears ahead of the site-packages directory where Conda installs packages, henceforth the version of a package available via Colab will take precedence over any version of the same package installed via Conda.

**Installing packages**

Remember to include the --yes flag when installing packages to avoid getting prompted to confirm the package plan.

In [11]:
!conda install --channel conda-forge featuretools --yes
!conda install -c conda-forge py2neo --yes
!conda install -c creditx ipython-cypher --yes


# >>>>>>>>>>>>>>>>>>>>>> ERROR REPORT <<<<<<<<<<<<<<<<<<<<<<

    Traceback (most recent call last):
      File "/usr/local/lib/python3.6/site-packages/conda/exceptions.py", line 1079, in __call__
        return func(*args, **kwargs)
      File "/usr/local/lib/python3.6/site-packages/conda/cli/main.py", line 84, in _main
        exit_code = do_call(args, p)
      File "/usr/local/lib/python3.6/site-packages/conda/cli/conda_argparse.py", line 83, in do_call
        return getattr(module, func_name)(args, parser)
      File "/usr/local/lib/python3.6/site-packages/conda/cli/main_install.py", line 20, in execute
        install(args, parser, 'install')
      File "/usr/local/lib/python3.6/site-packages/conda/cli/install.py", line 116, in install
        if context.use_only_tar_bz2:
      File "/usr/local/lib/python3.6/site-packages/conda/base/context.py", line 714, in use_only_tar_bz2
        import conda_package_handling.api
      File "/usr/local/lib/python3.6/site-packages/conda_packag

**Import**

In [12]:
from py2neo import Graph,Node,Relationship



**Connect to Neo4J Sandbox**

In [13]:
# Change the line of code below to use your Bolt URL, and Password of your Sandbox.
# graph = Graph("<Bolt URL>", auth=("neo4j", "<Password>"))
graph = Graph("bolt://3.238.79.232:7687", auth=("neo4j", "pronoun-accessories-airport"))